In [1]:
from llama_index.llms.mistralai import MistralAI
from llama_index.core.llms import ChatMessage
from mistralai import Mistral
import os

MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
model = "mistral-large-latest"

client = Mistral(api_key=MISTRAL_API_KEY)

Intent detection

In [2]:
def detect_intent(user_input):
    response = client.classifiers.classify(
    model="ft:classifier:ministral-3b-latest:82f3f89c:20250422:agro-intent-clf:a0b2cfa8",
    inputs=[user_input],
    )
    scores = response.results[0]['intent'].scores
    predicted_label = max(scores, key=scores.get)

    return predicted_label

detect_intent("How do I know if my tomato is sick ?")

'disease_diagnosis'

Vector search

In [3]:
from utils import initialize_qdrant

qdrant_client = initialize_qdrant()

Loaded 143 document embeddings for technical_reports
Document embeddings loaded successfully


In [21]:
import numpy as np

def score_vector_search(query, collection_name="technical_reports"):
    query_embedding = client.embeddings.create(
        model="mistral-embed",
        inputs=[query],
    ).data[0].embedding

    response = qdrant_client.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=30,
    )

    file_names = np.array([score.payload['file_name'].replace('.md', '') for score in response.points])
    scores_arr = np.array([score.score for score in response.points])

    unique_files = np.unique(file_names)
    scores_arr_norm = (scores_arr - np.nanmin(scores_arr)) / np.ptp(scores_arr) if np.ptp(scores_arr) > 0 else np.ones_like(scores_arr)

    max_scores_by_files = {str(file): scores_arr_norm[file_names == file].max() for file in unique_files}

    return max_scores_by_files

scores = score_vector_search("How do I know if my tomato is sick ?")
scores

{'2022_fiche-technique_environnement-2_nitrates': np.float64(0.0773110944521891),
 '2022_fiche-technique_presentation-generale': np.float64(0.6346333479492641),
 '2022_fiche-technique_sante-animaux-1_paquet-hygiene': np.float64(0.3223579544299179),
 '2022_fiche-technique_sante-animaux-3_ESST': np.float64(0.8302436619876772),
 '2022_fiche-technique_sante-animaux-4_identification': np.float64(0.11215354474649615),
 '2022_fiche-technique_sante-vegetaux-1_utilisation-PPP': np.float64(0.6059350915699976),
 '2022_fiche-technique_sante-vegetaux-2_paquet-hygiene': np.float64(0.6323755517108296),
 '2023_fiche-technique_BCAE7_rotation': np.float64(0.31559470383279503),
 '2023_fiche-technique_presentation-generale': np.float64(0.49884072436573423),
 '2023_fiche-technique_sante-vegetaux-1_utilisation-PPP': np.float64(0.9790109814903912),
 '2023_fiche-technique_sante-vegetaux-2_paquet-hygiene': np.float64(1.0),
 '2024_fiche-technique_BCAE7_rotation': np.float64(0.4036733569735661),
 '2024_fiche-tec

BM25 search

In [5]:
from utils import initialize_bm25

bm25 = initialize_bm25()

[nltk_data] Downloading package punkt to /home/estienne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
import nltk
import os

file_names = os.listdir("../../data/txt/technical_reports")

def score_bm25(query, top_k=30):
    tokenized_query = nltk.word_tokenize(query.lower())
    scores = bm25.get_scores(tokenized_query)

    scores_dict = dict(zip(file_names, scores))
    top_files = sorted(scores_dict.items(), key=lambda x: x[1], reverse=True)[:top_k]
    scores = np.array([score for _, score in top_files])

    bm25_norm = (scores - np.nanmin(scores)) / np.ptp(scores)
    norm_scores_dict = dict(zip([file.replace('.txt', '') for file, _ in top_files], bm25_norm))
    
    return norm_scores_dict

score_bm25("How do I know if my tomato is sick ?")

{'2022_fiche-technique_environnement-1_oiseaux-sauvages-habitats': np.float64(1.0),
 '2024_fiche-technique_BCAE8_biodiversite': np.float64(0.9099175712690547),
 '2022_fiche-technique_presentation-generale': np.float64(0.8473319764353349),
 '2022_fiche-technique_environnement-2_nitrates': np.float64(0.8424240240181591),
 '2022_fiche-technique_BCAE1_bande-tampon': np.float64(0.8341761022982922),
 '2024_fiche-technique_environnement-2_nitrates': np.float64(0.8197831308667083),
 '2022_fiche-technique_sante-animaux-1_paquet-hygiene': np.float64(0.801946336433694),
 '2023_fiche-technique_BCAE1_maintien-prairies-permanentes': np.float64(0.7814109494570135),
 '2024_fiche-technique_environnement-3_oiseaux_sauvages-habitats': np.float64(0.7808655207518197),
 '2023_fiche-technique_sante-vegetaux-1_utilisation-PPP': np.float64(0.7802726174229754),
 '2023_fiche-technique_BCAE8_biodiversite': np.float64(0.7652721395275097),
 '2024_fiche-technique_presentation-generale': np.float64(0.6380179581532436

Fusion document ranking

In [53]:
def score_fusion(query,top_k=5):
    vector_scores = score_vector_search(query)
    bm25_scores = score_bm25(query)

    all_files = set(vector_scores.keys()).union(set(bm25_scores.keys()))
    fusion_scores = {file: (vector_scores.get(file, 0) + bm25_scores.get(file, 0)) / 2 for file in all_files}

    top_files = sorted(fusion_scores.items(), key=lambda x: x[-1], reverse=True)[:top_k]
    top_files_dict = {file + ".md": score for file, score in top_files}
    return top_files_dict

score_fusion("Quelle est la législation sur les OGM ?")

{'2023_fiche-technique_sante-vegetaux-2_paquet-hygiene.md': np.float64(0.76630437896629),
 '2022_fiche-technique_sante-animaux-1_paquet-hygiene.md': np.float64(0.7552018219241939),
 '2022_fiche-technique_sante-vegetaux-2_paquet-hygiene.md': np.float64(0.7081201355625364),
 '2023_fiche-technique_sante-animaux-1_paquet-hygiene.md': np.float64(0.6908456257628153),
 '2024_fiche-technique_sante-animaux-1_paquet-hygiene.md': np.float64(0.6426470134379927)}

In [ ]:
def retrieve_documents(user_query):
    scores = score_fusion(user_query)
    documents = {k: v for k, v in scores.items()}
    return documents

retrieve_documents("Quelle est la législation sur les OGM ?")

In [89]:
from qdrant_client.http import models

def vector_search(query, file_names, collection_name="technical_reports"):
    query_embedding = client.embeddings.create(
        model="mistral-embed",
        inputs=[query],
    ).data[0].embedding

    response = qdrant_client.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=2,
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="file_name",
                    match=models.MatchAny(any=
                        file_names
                    )
                )
            ]
        )
    )

    return response.points

import os

files = os.listdir("../../data/md/technical_reports")

vector_search("Quelle est la législation sur les OGM ?", files)

[ScoredPoint(id='cb879bba-69ad-4d8c-a815-ce791526bc09', version=0, score=0.8000970153803234, payload={'text': " est interdit, toutefois le préfet peut, par décision motivée, autoriser un agriculteur à procéder au labour de la bande tampon en raison de son infestation par une espèce invasive listée en annexe de la présente fiche ; dans tous les cas, un travail superficiel du sol est autorisé,\n- dans le cas d'une parcelle en prairie ou pâturage, le pâturage de la bande tampon est autorisé, sous réserve du respect des règles d'usage pour l'accès des animaux au cours d'eau,\n- la fauche ou le broyage sont autorisés sur une largeur maximale de 20 mètres sur les parcelles enherbées déclarées en jachère,\n- les amendements alcalins (calciques et magnésiens) sont autorisés.\n\n# GRILLE BCAE - Bandes tampons le long des cours d'eau (Métropole) \n\n| Points de contrale | Anomalies | Système d'avertissement précoce |  | Réduction |\n| :--: | :--: | :--: | :--: | :--: |\n|  |  | Applicable? | Dél

Predict plant disease

In [69]:
from plant_disease.disease_prediction import predict_from_image

def predict_image(image):    
    contents = image.read()
    results = predict_from_image(image_data=contents)
    
    return results

with open("40285dce-33de-4a59-82f4-2eb1d6d38469___RS_LB 4929.JPG", "rb") as img_file:
    result = predict_image(img_file)
result

{'prediction': np.str_('Potato___Late_blight'),
 'confidence': 99.99701976776123,
 'top_predictions': [{'disease': np.str_('Potato___Late_blight'),
   'confidence': 99.99701976776123},
  {'disease': np.str_('Tomato_Late_blight'),
   'confidence': 0.002857758227037266},
  {'disease': np.str_('Potato___Early_blight'),
   'confidence': 7.023748480605718e-05}]}

In [90]:
from mistralai.models import UserMessage

def chat(messages):

    user_query = messages[-1].content
    intent = detect_intent(user_query)

    if intent == 'policy_help':
        context_docs = retrieve_documents(user_query)
        context = vector_search(user_query, context_docs.keys())

        context_text = "\n\n".join([f"Nom du document :{doc.payload['file_name']}. Date du document :{doc.payload['date']}.\nContenu du document :\n{doc.payload['text']}" for doc in context])

        message = UserMessage(
            content=f"Context: {context_text}\n\n{user_query}"
        )

        response = client.chat.complete(
            model=model,
            messages=[message],
            max_tokens=1000,
            temperature=0.1,
        )

        return response.choices[0].message.content

    elif intent == 'market_question':
        pass
    elif intent == 'disease_diagnosis':
        return "Please upload an image of the plant leaf for diagnosis."


    elif intent == 'weather_management':
        pass

    else:
        pass

chat([
    ChatMessage(
        role="user",
        content="Quelle est la législation sur les OGM ?"
    )
])


"La législation sur les organismes génétiquement modifiés (OGM) en Europe est principalement régie par plusieurs règlements et directives de l'Union européenne. Voici un aperçu des principaux textes législatifs :\n\n1. **Règlement (CE) n° 1829/2003** :\n   - Ce règlement concerne les denrées alimentaires et les aliments pour animaux génétiquement modifiés.\n   - Il établit des procédures pour l'autorisation des OGM dans l'alimentation humaine et animale.\n   - Il impose des exigences en matière d'étiquetage et de traçabilité pour les produits contenant des OGM.\n\n2. **Règlement (CE) n° 1830/2003** :\n   - Ce règlement concerne la traçabilité et l'étiquetage des OGM et des produits dérivés d'OGM destinés à être mis sur le marché.\n   - Il impose des obligations de traçabilité pour les OGM et les produits dérivés d'OGM à toutes les étapes de la chaîne de production et de distribution.\n\n3. **Directive 2001/18/CE** :\n   - Cette directive régit la dissémination volontaire d'OGM dans l'e

In [23]:
detect_intent("Comment mettre en place une culture de tomates en Normandie ?")

'other'